In [ ]:
import os
import tarfile
import urllib
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()
    
    


In [ ]:
fetch_housing_data()

In [ ]:
import pandas as pd

#데이터 담긴 판다스 데이터 프레임 객체 반환
def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)



In [ ]:
housing = load_housing_data()
housing.head()

In [ ]:
housing.info()

In [ ]:
housing["ocean_proximity"].value_counts()

In [ ]:
#%matplotlib inline
import matplotlib.pyplot as plt
housing.hist(bins=50, figsize=(20, 15))
plt.show()

In [ ]:
#Test Set 만들기
import numpy as np

def split_train_test(data, test_ratio):
    shuffled_indices = np.random.permutation(len(data))
    test_set_size = int(len(data) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return data.iloc[train_indices], data.iloc[test_indices]

In [ ]:
train_set, test_set = split_train_test(housing, 0.2)

In [ ]:
len(train_set)

In [ ]:
len(test_set)

In [ ]:
from zlib import crc32

def test_set_check(identifier, test_ratio):
    return crc32(np.int64(identifier)) & 0xffffffff < test_ratio * 2**32

In [ ]:
def split_train_test_by_id(data, test_ratio, id_column):
    ids = data[id_column]
    in_test_set = ids.apply(lambda id_: test_set_check(id_, test_ratio))
    return data.loc[~in_test_set], data.loc[in_test_set]

In [ ]:
housing_with_id = housing.reset_index()
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "index")

In [ ]:
housing_with_id["id"] = housing["longitude"] * 1000 + housing["latitude"]
train_set, test_set = split_train_test_by_id(housing_with_id, 0.2, "id")

In [ ]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(housing, test_size = 0.2, random_state=42)

In [ ]:
housing["income_cat"] = pd.cut(housing["median_income"], bins = [0., 1.5, 3.0, 4.5, 6., np.inf],
                              labels=[1,2,3,4,5])

In [ ]:
housing["income_cat"].hist()

In [ ]:
#SKELARN BASED stratified shufflesplit 
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state = 42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index]
    strat_test_set = housing.loc[test_index]

In [ ]:
# 테스트 세트에서 소득 카테고리 비율 
strat_test_set["income_cat"].value_counts() / len(strat_test_set)

In [ ]:
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

In [ ]:
# 훈련 세트 복사본
housing = strat_train_set.copy()


In [ ]:
#지리 데이터 시각화
housing.plot(kind="scatter", x="longitude", y="latitude")

In [ ]:
# alpha옵션을 0.1로 주어서 데이터 포인트가 밀집된 영역 보기
housing.plot(kind="scatter", x="longitude", y="latitude", alpha=0.1)

In [ ]:
housing.plot(
    kind="scatter", 
    x="longitude", 
    y="latitude", 
    alpha=0.4, 
    s=housing["population"]/100, label="population", figsize=(10,7),
            c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True, sharex=False)
plt.legend()

In [ ]:
#데이터셋이 많이 크진 않으니, 표준 상관계수를 이용

corr_matrix = housing.corr()

#1 = positive relationships, -1 = negative relationships
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
# 상관관계 확인을 위해 산점도로 파악
#중간주택 가격과 상관관계가 높은 특성 몇 개만 추려서
from pandas.plotting import scatter_matrix

attributes = ["median_house_value", "median_income", "total_rooms", "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12,8))

In [ ]:
#중간 소득과 중간 주택 가격을 예측하는게 유용할 것 같아서 확대
housing.plot(kind="scatter", x="median_income", y="median_house_value", alpha=0.1)

- 위 그래프는 상관관계가 아주 강함을 보여준다
- 가격 제한이었던 500000불이 수평선으로 잘 보인다
- 수평으로 보이는 부분은 학습하는 과정에서 좋지 않기 때문에 정제해줄 필요가 있다. 



In [ ]:
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_room"] = housing["total_bedrooms"] / housing["total_rooms"]
housing["population_per_household"] = housing["population"]/housing["households"]

In [ ]:
corr_matrix=housing.corr()

In [ ]:
corr_matrix["median_house_value"].sort_values(ascending=False)

- median_house_value와 medina_income의 상관관계가 0.68으로 나쁘지 않다
- bedrooms/room 비율이 낮은 집일수록 집이 비싼 경향


In [ ]:
housing = strat_train_set.drop("median_house_value", axis=1)
housing_labels = strat_train_set["median_house_value"].copy()

#### 머신러닝 알고리즘은 누락 특성을 다루지 못하므로 이를 처리할 함수를 만든다
> total_bedroom특성에 값이 없는 경우를 보았는데 이를 고치자
- 해당구역 제거
- 전체 특성을 삭제
- 어떤 값으로 채우기 0, 평균, 중간값 등



In [ ]:
housing.dropna(subset=["total_bedrooms"])
housing.drop("total_bedrooms", axis=1)
median= housing["total_bedrooms"].median()
housing["total_bedrooms"].fillna(median, inplace=True)

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy="median")

- sklearn의 SimpleImputer는 누락된 값을 손쉽게 다루도록 해준다. 
- 위 코드는 눌가 값을 중간값으로 대체한다고 지정한다


In [ ]:
housing_num = housing.drop("ocean_proximity", axis=1)

In [ ]:
imputer.fit(housing_num)

- imputer는 특성의 중간값을 계산해서 그 결과를 객체의 statistics_ 속성에 저장한다
- total_bedrooms 특성에만 누락된 값이 있지만, 나중에 시스템이 서비스될 때 새로운 데이터에서 어떤 값이 누락될지 확신할 수 없으므로 모든 수치형 특성에 imputer를 적용하는게 바람직하다


In [ ]:
imputer.statistics_

In [ ]:
housing_num.median().values

In [ ]:
X = imputer.transform(housing_num)

In [ ]:
hosing_tr = pd.DataFrame(X, columns=housing_num.columns, index=housing_num.index)

In [ ]:
# 텍스트, 범주형특성 다루기
housing_cat = housing[["ocean_proximity"]]
housing_cat.head(10)

- 대부분 머신러닝 알고리즘은 숫자를 다룬다
- 텍스트를  숫자로 바꾸기 위해 sklearn의 OrdinalEncoder클래스를 사용한다

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
housing_cat_encoded = ordinal_encoder.fit_transform(housing_cat)
housing_cat_encoded[:10]

In [ ]:
ordinal_encoder.categories_

- 범주형 데이터의 경우, 인접 인덱스가 큰 상관관계를 가진다고 머신러닝이 학습하는 경향이 있다. 
- 우리의 범주형 데이터는 0, 4번 인덱스가 오히려 큰 상관관계를 갖기때문에 변형 시킬 필요가 있다(one-hot encoding)

In [ ]:
#sklearn의 onehotencoder
from sklearn.preprocessing import OneHotEncoder
cat_encoder = OneHotEncoder()
housing_cat_1hot = cat_encoder.fit_transform(housing_cat)
housing_cat_1hot

- SciPy의 희소행렬로 결과가 나오는데, 수천 개의 카테고리가 있는 범주형 데이터의 경우에는 아주 효율적이다.(0이 아닌 원소의 "위치"만 기억하기 때문)
- 넘파이 배열로 바꾸려면 toarray()메소드를 호출하면 된다.

In [ ]:
housing_cat_1hot.toarray()

In [ ]:
#인코더의 categories_ 인스턴스 변수를 사용해 카테고리 리스트를 얻을 수 있다
cat_encoder.categories_